In [48]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from timm import create_model
from sklearn.model_selection import train_test_split
from PIL import Image

In [49]:
class CelebASpoofDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [62]:
def prepare_data_from_multiple_folders(base_dirs, test_size=0.2, val_size=0.1, valid_extensions=(".jpg", ".jpeg", ".png")):

    image_paths = []
    labels = []

    for base_dir in base_dirs:
        for label_name, label_idx in [("live", 0), ("spoof", 1)]:
            folder = os.path.join(base_dir, label_name)
            if not os.path.exists(folder):
                print(f"Папка {folder} нет.")
                continue
            for img_file in os.listdir(folder):
                if img_file.lower().endswith(valid_extensions):
                    image_paths.append(os.path.join(folder, img_file))
                    labels.append(label_idx)

    train_paths, test_paths, train_labels, test_labels = train_test_split(
        image_paths, labels, test_size=test_size, random_state=42, stratify=labels
    )
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        train_paths, train_labels, test_size=val_size, random_state=42, stratify=train_labels
    )

    return train_paths, val_paths, test_paths, train_labels, val_labels, test_labels


In [59]:
def create_mobilevit_model(num_classes=2, pretrained=True):
    model = create_model("mobilevitv2_175", pretrained=pretrained) 
    
    dummy_input = torch.randn(1, 3, 224, 224)  
    features = model.forward_features(dummy_input) 
    num_features = features.shape[1] * features.shape[2] * features.shape[3] 

    model.head = nn.Sequential(
        nn.Flatten(), 
        nn.Linear(num_features, num_classes)  
    )
    return model

In [57]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"):
    model.to(device)
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100. * correct / total

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_acc = 100. * correct / total

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_mobilevitv2_175.pth")

    print(f"Best Validation Accuracy: {best_acc:.2f}%")

In [65]:
if __name__ == "__main__":
    base_dirs = [
        "C:/Users/annam/Documents/Проекты/Диплом/Данные 2/CelebA_Spoof/Data/train/1/",
        "C:/Users/annam/Documents/Проекты/Диплом/Данные 2/CelebA_Spoof/Data/train/2/",
        "C:/Users/annam/Documents/Проекты/Диплом/Данные 2/CelebA_Spoof/Data/train/3/"
    ]
    
    
    batch_size = 8
    num_epochs = 5
    learning_rate = 0.001
    device = "cuda" if torch.cuda.is_available() else "cpu"

    train_paths, val_paths, test_paths, train_labels, val_labels, test_labels = prepare_data_from_multiple_folders(base_dirs)

    input_size = (224, 224) 

    transform = transforms.Compose([
        transforms.Resize(input_size),  
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    train_dataset = CelebASpoofDataset(train_paths, train_labels, transform=transform)
    val_dataset = CelebASpoofDataset(val_paths, val_labels, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = create_mobilevit_model(num_classes=2, pretrained=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs, device=device)

    print("Training completed. The best model is saved as 'best_mobilevitv2_175.pth'.")

Epoch 1/5, Train Loss: 17.4477, Train Acc: 88.89%, Val Loss: 0.4008, Val Acc: 93.33%
Epoch 2/5, Train Loss: 27.5243, Train Acc: 88.89%, Val Loss: 1.8773, Val Acc: 86.67%
Epoch 3/5, Train Loss: 16.7612, Train Acc: 94.81%, Val Loss: 9.4450, Val Acc: 86.67%
Epoch 4/5, Train Loss: 7.2905, Train Acc: 96.30%, Val Loss: 8.8427, Val Acc: 86.67%
Epoch 5/5, Train Loss: 9.0267, Train Acc: 94.07%, Val Loss: 0.6234, Val Acc: 93.33%
Best Validation Accuracy: 93.33%
Training completed. The best model is saved as 'best_mobilevitv2_175.pth'.


In [66]:
def test_model(model, test_loader, device="cuda"):
    """
    Проверка точности модели на тестовых данных.
    """
    model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_acc = 100. * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%")

In [67]:
test_dataset = CelebASpoofDataset(test_paths, test_labels, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
model.load_state_dict(torch.load("best_mobilevitv2_175.pth"))

test_model(model, test_loader, device=device)

C:\Users\annam\AppData\Local\Temp\ipykernel_113336\3793627334.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_mobilevitv2_175.pth"

Test Accuracy: 89.47%
